In [1]:
import numpy as np
from sklearn import preprocessing

In [2]:
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter=',')
unscaled_inputs = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

In [3]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove =[]
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
unscaled_inputs_equal_priors = np.delete(unscaled_inputs, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [4]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [5]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [6]:
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count :train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count :train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count)

1798.0 3579 0.502374965074043
216.0 447 0.48322147651006714
223.0 448 0.49776785714285715


In [7]:
np.savez('Audiobooks_data_train', inputs = train_inputs, targets = train_targets)
np.savez('Audiobooks_data_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test', inputs = test_inputs, targets = test_targets)

In [8]:
import tensorflow as tf

In [10]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs = npz['inputs'].astype(np.float)
validation_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs = npz['inputs'].astype(np.float)
test_targets = npz['targets'].astype(np.int)

/tmp/ipykernel_13374/4061884859.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs = npz['inputs'].astype(np.float)
/tmp/ipykernel_13374/4061884859.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_ta

In [14]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(output_size, activation ='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

max_epochs = 100
batch_size = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_inputs, 
          train_targets,
          batch_size = batch_size, 
          epochs = max_epochs, 
          callbacks = [early_stopping],
          validation_data = (validation_inputs, validation_targets),
          verbose =2)

Epoch 1/100
36/36 - 1s - loss: 0.5290 - accuracy: 0.8075 - val_loss: 0.4263 - val_accuracy: 0.8725
Epoch 2/100
36/36 - 0s - loss: 0.3672 - accuracy: 0.8715 - val_loss: 0.3625 - val_accuracy: 0.8747
Epoch 3/100
36/36 - 0s - loss: 0.3167 - accuracy: 0.8826 - val_loss: 0.3357 - val_accuracy: 0.8814
Epoch 4/100
36/36 - 0s - loss: 0.2963 - accuracy: 0.8888 - val_loss: 0.3168 - val_accuracy: 0.8926
Epoch 5/100
36/36 - 0s - loss: 0.2798 - accuracy: 0.8930 - val_loss: 0.3065 - val_accuracy: 0.8881
Epoch 6/100
36/36 - 0s - loss: 0.2699 - accuracy: 0.8947 - val_loss: 0.2937 - val_accuracy: 0.8949
Epoch 7/100
36/36 - 0s - loss: 0.2648 - accuracy: 0.8972 - val_loss: 0.2851 - val_accuracy: 0.9038
Epoch 8/100
36/36 - 0s - loss: 0.2606 - accuracy: 0.8997 - val_loss: 0.2807 - val_accuracy: 0.8993
Epoch 9/100
36/36 - 0s - loss: 0.2542 - accuracy: 0.9011 - val_loss: 0.2740 - val_accuracy: 0.9083
Epoch 10/100
36/36 - 0s - loss: 0.2486 - accuracy: 0.9036 - val_loss: 0.2721 - val_accuracy: 0.9083
Epoch 11/

In [15]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 8ms/step - loss: 0.2465 - accuracy: 0.9085
